#Imports and Helper Functions

##Installations

In [ ]:
#@title Install Spatial Tracker and Download Zeo Depth Model

%cd /content
!git clone https://github.com/henry123-boy/SpaTracker
%cd /content/SpaTracker
!pip install av
!pip install timm==0.6.7 flow_vis
!pip install -r requirements.txt

!mkdir /content/checkpoints
!gdown -O /content/checkpoints/spaT_final.pth 18YlG_rgrHcJ7lIYQWfRz_K669z6FdmUX

!mkdir -p /content/models/monoD/zoeDepth/ckpts
!wget -O /content/models/monoD/zoeDepth/ckpts/dpt_beit_large_384.pt https://github.com/isl-org/MiDaS/releases/download/v3_1/dpt_beit_large_384.pt
!wget -O /content/models/monoD/zoeDepth/ckpts/ZoeD_M12_K.pt https://github.com/isl-org/ZoeDepth/releases/download/v1.0/ZoeD_M12_K.pt
!wget -O /content/models/monoD/zoeDepth/ckpts/ZoeD_M12_NK.pt https://github.com/isl-org/ZoeDepth/releases/download/v1.0/ZoeD_M12_NK.pt

%cd /content

In [ ]:
#@title Install Depth Anything V2

%cd /content
!git clone https://github.com/DepthAnything/Depth-Anything-V2
%cd /content/Depth-Anything-V2
!pip install -r requirements.txt

!mkdir /content/checkpoints
!wget -O /content/checkpoints/depth_anything_v2_vitl.pth https://huggingface.co/depth-anything/Depth-Anything-V2-Large/resolve/main/depth_anything_v2_vitl.pth?download=true

%cd /content

In [ ]:
#@title Download Some Example

!gdown -O /content/SpaTracker/assets/butterfly.mp4 1BDtvfrvbzEFY84XJPp62Dq1PujIpbOK_
!gdown -O /content/SpaTracker/assets/butterfly.png 1hlAGFony7LzpLcEAoGLiNaY3zxfiN_bW
!gdown -O /content/SpaTracker/assets/sintel_bandage.mp4 1iL5Qs5ea8r9nFwgVC6fusFyBfFQDICyo
!gdown -O /content/SpaTracker/assets/sintel_bandage.png 1_cL3m_1bW6aFwhxRPr_vGkdizwJuObbH

##Imports

In [ ]:
import os, sys, gc
from typing import Optional
from PIL import Image
import numpy as np
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
from moviepy.editor import ImageSequenceClip

%cd /content/SpaTracker/

from models.spatracker.predictor import SpaTrackerPredictor
from models.spatracker.utils.visualizer import Visualizer
from models.monoD.zoeDepth.models.builder import build_model
from models.monoD.zoeDepth.utils.config import get_config

%cd /content/Depth-Anything-V2

from depth_anything_v2.dpt import DepthAnythingV2

%cd /content

/content/SpaTracker
/content/Depth-Anything-V2
/content


##Helpers

In [ ]:
#@title Memory Helpers

def free_memory():
    gc.collect()
    torch.cuda.empty_cache()

def clear_gpu_memory():
    torch.cuda.empty_cache()
    if torch.cuda.is_available():
        torch.cuda.ipc_collect()
        torch.cuda.synchronize()

In [ ]:
#@title Video Helpers

def load_video(
    vid_path: str,
    downsample: float = 1.0,
    frame_interval: int = 1,
    max_frames: Optional[int] = None
) -> torch.Tensor:
    cap = cv2.VideoCapture(vid_path)
    if not cap.isOpened():
        raise IOError(f"Error opening video file: {vid_path}")

    try:
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        if max_frames:
            total_frames = min(total_frames, max_frames)

        frames = []
        for i in range(0, total_frames, frame_interval):
            cap.set(cv2.CAP_PROP_POS_FRAMES, i)
            ret, frame = cap.read()
            if not ret:
                break

            frame = process_frame(frame, downsample)
            frames.append(frame)

        if not frames:
            raise RuntimeError("No frames could be read from the video")

        return torch.cat(frames, dim=0)

    finally:
        cap.release()

def process_frame(frame: np.ndarray, downsample: float) -> torch.Tensor:
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_tensor = torch.from_numpy(frame).permute(2, 0, 1).float().unsqueeze(0)

    if downsample != 1.0:
        frame_tensor = F.interpolate(
            frame_tensor,
            scale_factor=downsample,
            mode='bilinear',
            align_corners=False
        )

    return frame_tensor

def load_segmentation_mask(seg_path, H, W):
    if os.path.exists(seg_path):
        segm_mask = np.array(Image.open(seg_path))
    else:
        segm_mask = np.ones((H, W), dtype=np.uint8)
        print("No segmentation mask provided. Computing tracks in whole image.")

    if segm_mask.ndim == 3:
        segm_mask = (segm_mask[..., :3].mean(axis=-1) > 0).astype(np.uint8)

    return cv2.resize(segm_mask, (W, H), interpolation=cv2.INTER_NEAREST)

In [ ]:
#@title Visualization Helpers

def visualize_results(video, pred_tracks, pred_visibility, outdir, vid_name, fps_vis, len_track, point_size):
    vis = Visualizer(save_dir=outdir, grayscale=True,
                     fps=fps_vis, pad_value=0, linewidth=point_size,
                     tracks_leave_trace=len_track)
    return vis.visualize(
        video=video,
        tracks=pred_tracks[..., :2],
        visibility=pred_visibility,
        filename=f"{vid_name}_spatracker"
    )

In [ ]:
#@title Saving helpers

def save_results(video, video_vis, pred_tracks, outdir, vid_name, model_name):
    img0 = video_vis[0,0].permute(1,2,0).detach().cpu().numpy()
    cv2.imwrite(os.path.join(outdir, f'{vid_name}_ref_query.png'), img0[:,:,::-1])

    tracks_vis = pred_tracks[0].detach().cpu().numpy()
    np.save(os.path.join(outdir, f'{vid_name}_{model_name}_tracks.npy'), tracks_vis)

    wide_list = [wide[0].permute(1, 2, 0).cpu().numpy() for wide in video.unbind(1)]
    clip = ImageSequenceClip(wide_list, fps=60)
    save_path = os.path.join(outdir, f'{vid_name}_vid.mp4')
    clip.write_videofile(save_path, codec="libx264", fps=25, logger=None)
    print(f"Original Video saved to {save_path}")

def save_3d_trajectories(pred_tracks, video, outdir, vid_name):
    T, N, _ = pred_tracks[0].shape
    H, W = video[0].shape[-2:]
    xyzt = pred_tracks[0].cpu().numpy()

    intr = np.array([[W, 0.0, W//2],
                     [0.0, W, H//2],
                     [0.0, 0.0, 1.0]])

    xyztVis = xyzt.copy()
    xyztVis[..., 2] = 1.0
    xyztVis = np.linalg.inv(intr[None, ...]) @ xyztVis.reshape(-1, 3, 1)
    xyztVis = xyztVis.reshape(T, -1, 3)
    xyztVis[..., 2] *= xyzt[..., 2]

    pred_tracks2d = pred_tracks[0][:, :, :2]
    pred_tracks2dNm = pred_tracks2d.clone()
    pred_tracks2dNm[..., 0] = 2*(pred_tracks2dNm[..., 0] / W - 0.5)
    pred_tracks2dNm[..., 1] = 2*(pred_tracks2dNm[..., 1] / H - 0.5)

    color_interp = torch.nn.functional.grid_sample(video[0], pred_tracks2dNm[:,:,None,:], align_corners=True)
    color_interp = color_interp[:, :, :, 0].permute(0,2,1).cpu().numpy().astype(np.uint8)

    colored_pts = np.concatenate([xyztVis, color_interp], axis=-1)
    np.save(f'{outdir}/{vid_name}_3d.npy', colored_pts)
    print(f"3D colored tracks saved to {outdir}/{vid_name}_3d.npy")

#Implementation

##Models

In [ ]:
#@title Depth Model

class DepthAnythingV2Wrapper(nn.Module):
    def __init__(self, encoder='vitl', features=256, out_channels=[256, 512, 1024, 1024]):
        super(DepthAnythingV2Wrapper, self).__init__()
        self.model = DepthAnythingV2(encoder=encoder, features=features, out_channels=out_channels)
        self.model.load_state_dict(torch.load('/content/checkpoints/depth_anything_v2_vitl.pth', map_location='cpu'))
        self.model = self.model.to(device).eval()

    def infer(self, rgbs):
        with torch.no_grad():
            batch_size, channels, height, width = rgbs.shape
            depth_maps = []

            for i in range(batch_size):
                img = rgbs[i].permute(1, 2, 0).cpu().numpy()
                img = (img * 255).astype(np.uint8)
                depth = self.model.infer_image(img)

                depth_tensor = torch.from_numpy(depth).unsqueeze(0).unsqueeze(0)
                depth_maps.append(depth_tensor)

            return torch.cat(depth_maps, dim=0).to(rgbs.device)

class MonoDEst(nn.Module):
    def __init__(self):
        super(MonoDEst, self).__init__()
        self.model = self._build_model()
        self.metric3d = build_model(get_config("zoedepth_nk", "infer")).to(device).eval()

    def _build_model(self):
        return DepthAnythingV2Wrapper(encoder='vitl', features=256, out_channels=[256, 512, 1024, 1024])

    def infer(self, rgbs, scale=None, shift=None):
        depth_map = self.model.infer(rgbs)
        metric_dp = self.metric3d.infer(rgbs[:20])
        metric_dp_inv = 1 / metric_dp
        dp_0_rel = depth_map[:20]
        scale, shift = np.polyfit(dp_0_rel.view(-1).cpu().numpy(),
                                  metric_dp_inv.view(-1).cpu().numpy(), 1)
        depth_map = depth_map * scale + shift
        return (1 / depth_map).clamp(0.01, 65)

##Inference

In [ ]:
def setup_environment(root, vid_name, outdir):
    os.makedirs(outdir, exist_ok=True)
    vid_path = os.path.join(root, f'{vid_name}.mp4')
    seg_path = os.path.join(root, f'{vid_name}.png')
    return vid_path, seg_path

def load_and_process_data(vid_path, seg_path, downsample, fps):
    video = load_video(vid_path, downsample, frame_interval)
    _, _, H, W = video.shape
    segm_mask = load_segmentation_mask(seg_path, H, W)
    return video, segm_mask

def save_initial_images(video, segm_mask, outdir, vid_name):
    img0 = video[0].permute(1, 2, 0).detach().cpu().numpy()
    cv2.imwrite(os.path.join(outdir, f'{vid_name}_ref.png'), img0[:, :, ::-1])
    cv2.imwrite(os.path.join(outdir, f'{vid_name}_seg.png'), segm_mask * 255)

def initialize_models(interp_shape, seq_length, device):
    spatracker_predictor = SpaTrackerPredictor(
        checkpoint='/content/checkpoints/spaT_final.pth',
        interp_shape=interp_shape,
        seq_length=seq_length
    ).to(device)
    monodepth_model = MonoDEst().model
    monodepth_model.eval()
    return spatracker_predictor, monodepth_model

def process_video(spatracker_predictor, monodepth_model, video, segm_mask, query_frame, seq_length):
    pred_tracks, pred_visibility, T_Firsts = spatracker_predictor(
        video, video_depth=None, grid_size=40, backward_tracking=False,
        depth_predictor=monodepth_model, grid_query_frame=query_frame,
        segm_mask=torch.from_numpy(segm_mask).unsqueeze(0).unsqueeze(0),
        wind_length=seq_length
    )

    msk_query = (T_Firsts == query_frame)
    pred_tracks = pred_tracks.cpu()[:, :, msk_query.squeeze()]
    pred_visibility = pred_visibility.cpu()[:, :, msk_query.squeeze()]
    return pred_tracks, pred_visibility

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

root = '/content/SpaTracker/assets'
vid_name = 'butterfly'
outdir = './vis_results'
downsample = 1
frame_interval = 2
len_track = 3
fps_vis = 15
query_frame = 0
point_size = 3
seq_length = 12
interp_shape = (384, 512)

vid_path, seg_path = setup_environment(root, vid_name, outdir)
video, segm_mask = load_and_process_data(vid_path, seg_path, downsample, frame_interval)
save_initial_images(video, segm_mask, outdir, vid_name)

free_memory()

spatracker_predictor, monodepth_model = initialize_models(interp_shape, seq_length, device)

free_memory()

video = video.unsqueeze(0).to(device)
pred_tracks, pred_visibility = process_video(spatracker_predictor, monodepth_model, video, segm_mask, query_frame, seq_length)
video = video.cpu()

free_memory()

video_vis = visualize_results(video, pred_tracks, pred_visibility, outdir, vid_name, fps_vis, len_track, point_size)
save_results(video, video_vis, pred_tracks, outdir, vid_name, 'spatracker')
save_3d_trajectories(pred_tracks, video, outdir, vid_name)

del video, segm_mask, pred_tracks, pred_visibility, video_vis
free_memory()
clear_gpu_memory()